In [1]:
#!/usr/bin/env python

# Copyright (c) 2018-2021, Vijay Kandiah, Junrui Pan, Mahmoud Khairy, Scott Peverelle, Timothy Rogers, Tor M. Aamodt, Nikos Hardavellas
# Northwestern University, Purdue University, The University of British Columbia
# All rights reserved.

# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:

# 1. Redistributions of source code must retain the above copyright notice, this
#    list of conditions and the following disclaimer;
# 2. Redistributions in binary form must reproduce the above copyright notice,
#    this list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution;
# 3. Neither the names of Northwestern University, Purdue University,
#    The University of British Columbia nor the names of their contributors
#    may be used to endorse or promote products derived from this software
#    without specific prior written permission.

# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
# ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE
# LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR
# CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF
# SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS
# INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN
# CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.

import json
import csv
import pandas as pd
import numpy
import os 
import collections 
import glob
import sys
from os.path import dirname, basename, isfile, join
import shutil

In [2]:
rootdir = os.getcwd();
results_dir = rootdir + '/accelwattch_power_reports'
os.listdir(results_dir)

['QV100_volta_sass_hw',
 'power_ubench',
 'ubench_datatype_volta_sass_sim',
 'power_ubench_datatype_sass_sim',
 'TeslaV100_Incomplete_BadLCDInsts_volta_sass_hw',
 '^C',
 'TeslaV100_tuner_only_volta_sass_hw',
 'TeslaV100_BadLDC-Accelwattch_SASS_SIM',
 'volta_hw_sass_GPU_Microbench',
 'power_ubench_datatype_sass_sim_second']

In [3]:
configs = ["ubench_datatype_volta_sass_sim","power_ubench_datatype_sass_sim","TeslaV100_BadLDC-Accelwattch_SASS_SIM","power_ubench","TeslaV100_Incomplete_BadLCDInsts_volta_sass_hw","volta_hw_sass_GPU_Microbench"]
config = configs[0] #Just get the first config
print(f"Processing the accelwattch_power_reports/{config} logs")

Processing the accelwattch_power_reports/ubench_datatype_volta_sass_sim logs


In [4]:
power_counters = ['IBP,', 'ICP,', 'DCP,', 'TCP,', 'CCP,', 'SHRDP,', 'RFP,', 
                    'INTP,', 'FPUP,', 'DPUP,', 'INT_MUL24P,', 'INT_MUL32P,', 'INT_MULP,','INT_DIVP,', 'FP_MULP,',
                    'FP_DIVP,', 'FP_SQRTP,', 'FP_LGP,', 'FP_SINP,', 'FP_EXP,','DP_MULP,', 'DP_DIVP,', 'TENSORP,', 'TEXP,',                                  
                    'SCHEDP,', 'L2CP,', 'MCP,', 'NOCP,', 'DRAMP,','PIPEP,', 'IDLE_COREP,', 'CONSTP','STATICP','kernel_avg_power']
rootdir = os.getcwd();
results_dir = rootdir + '/accelwattch_results'

print("Collecting AccelWattch power results for " + config)
reportspath = rootdir + '/accelwattch_power_reports/' + config
power_dict = collections.OrderedDict()

In [5]:
benchmarks = []
for dirname in os.listdir(reportspath):
    benchmarks.append(dirname[:-4])
bad_benchmarks_list = [
    "cutlass_perf_test_k1",
    "cutlass_perf_test_k2",
    "cutlass_perf_test_k3",
    "parboil-mri-q",
    "dct8x8",
    "binomialOptions",
    "quasirandomGenerator",
    "kmeans-rodinia-3.1"
]
if "Bad" in config:
    for bad_benchmark in bad_benchmarks_list:
        if bad_benchmark in benchmarks:
            benchmarks.remove(bad_benchmark)
benchmarks



['BE_SP_INT_ADD',
 'BE_HP_FP_MUL',
 'BE_DP_FP_MUL',
 'BE_DP_FP_ADD',
 'BE_SP_FP_MUL',
 'BE_HP_FP_ADD',
 'BE_SP_FP_ADD']

In [6]:
kernel_name_map = {}
for benchmark in benchmarks:#[benchmarks[1]]:
    with open((reportspath + '/' + benchmark + '.log'), 'r') as f:
        x = f.readlines()
    kernels_encountered = 1
    unique_kernel_counts = {}
    kernelnameheader = 'kernel_name='
    baselines = collections.OrderedDict()
    
    for each in range(len(x)):
        x[each] = x[each].replace(" ", "")
    for each in range(len(x)):
        if (x[each].find(kernelnameheader) != -1):
            _,kernelname=x[each].strip().split('=')
            if kernelname not in unique_kernel_counts:
                #kernelname=kernelname.strip()
                unique_kernel_counts[kernelname] = 1
                benchmark_idx = (benchmark + '_k' + str(kernels_encountered))
                kernel_name_map[kernelname] = benchmark_idx
                power_dict[benchmark_idx] = collections.OrderedDict()
                kernels_encountered += 1
                
                baselines[benchmark_idx] = collections.OrderedDict()
                for counter in power_counters:
                    baselines[benchmark_idx][counter] = 0
            else:
                unique_kernel_counts[kernelname] += 1 
            # print(type(x))
            key,value = x[each+4].split('=')
            # print(key)
            value = value.rstrip('\n')
            baselines[kernel_name_map[kernelname]][(key)] += float(value)
            for stat in range(5,38):
                key,value = x[each+stat].split('=')
                # print(key,stat-4)
                value = value.rstrip('\n')
                baselines[kernel_name_map[kernelname]][(key[8:])] += float(value)
    # print(benchmark,baselines)
    bench_idx_to_kernelname_dict = {v:k for k,v in kernel_name_map.items()}
    if len(kernel_name_map) != len(bench_idx_to_kernelname_dict):
        print("Error in mapping kernels to given benchmark_idx")
        continue
    for benchmark_idx in baselines:
        kernelname = bench_idx_to_kernelname_dict[benchmark_idx]
        for each in power_counters:
            #power_dict[benchmark_idx][each] = float(baseline[each])
            power_dict[benchmark_idx][each] = float(baselines[benchmark_idx][each]) / float(unique_kernel_counts[kernelname])
        power_dict[benchmark_idx]['DRAMP,'] = power_dict[benchmark_idx]['DRAMP,'] + power_dict[benchmark_idx]['MCP,']
        power_dict[benchmark_idx]['L2CP,'] = power_dict[benchmark_idx]['L2CP,'] + power_dict[benchmark_idx]['NOCP,']
    if len(baselines) == 0:
        print("Warning: " + benchmark + " has no simulator data.") 
df = pd.DataFrame.from_dict(power_dict, orient='index').sort_index()
df


,"IBP,","ICP,","DCP,","TCP,","CCP,","SHRDP,","RFP,","INTP,","FPUP,","DPUP,",...,"SCHEDP,","L2CP,","MCP,","NOCP,","DRAMP,","PIPEP,","IDLE_COREP,",CONSTP,STATICP,kernel_avg_power
BE_DP_FP_ADD_k1,0.456432,0.0,1.507010,0.0,0.0,0.0,17.21460,0.655461,0.000000,32.4595,...,1.31651,1.396768,0.987541,0.923933,0.987541,1.214070,6.60296,32.3252,26.449000,122.942
BE_DP_FP_MUL_k1,0.446203,0.0,1.484780,0.0,0.0,0.0,16.82410,0.495841,0.000000,0.0000,...,1.29460,1.403267,0.981576,0.924685,0.981576,1.186870,6.89266,32.3252,25.953700,100.734
BE_HP_FP_ADD_k1,0.060785,0.0,1.142490,0.0,0.0,0.0,1.43205,0.686975,0.220053,0.0000,...,0.15513,1.056839,0.882068,0.702047,0.882068,0.161641,20.14730,32.3252,4.134845,62.700
BE_HP_FP_ADD_k2,0.860364,0.0,0.690624,0.0,0.0,0.0,31.79800,0.765365,42.232200,0.0000,...,2.48181,0.644605,0.746202,0.428537,0.746202,2.287490,12.78770,32.3252,16.726900,145.082
BE_HP_FP_MUL_k1,0.060785,0.0,1.142490,0.0,0.0,0.0,1.43205,0.686975,0.220053,0.0000,...,0.15513,1.056839,0.882068,0.702047,0.882068,0.161641,20.14730,32.3252,4.134845,62.700
BE_HP_FP_MUL_k2,0.650741,0.0,0.534336,0.0,0.0,0.0,24.20540,0.579499,0.104713,0.0000,...,1.89932,0.509888,0.703969,0.335329,0.703969,1.730400,9.71322,32.3252,21.990800,106.458
BE_SP_FP_ADD_k1,0.639322,0.0,1.061100,0.0,0.0,0.0,23.56510,1.133490,31.074700,0.0000,...,1.82730,1.003143,0.858578,0.660909,0.858578,1.700580,9.59843,32.3252,22.198400,127.304
BE_SP_FP_MUL_k1,0.635561,0.0,1.060760,0.0,0.0,0.0,23.34230,0.920919,0.000000,0.0000,...,1.81623,1.001694,0.858424,0.660231,0.858424,1.690570,9.64087,32.3252,22.113900,106.463
BE_SP_INT_ADD_k1,0.417835,0.0,1.603840,0.0,0.0,0.0,19.50470,12.558200,0.000000,0.0000,...,1.21321,1.432461,1.008480,0.966577,1.008480,1.110780,14.42360,32.3252,12.077500,98.807


In [7]:
import pprint

pprint.pprint(kernel_name_map)

{'_Z12PowerKernal1PK6__halfS1_PS_i': 'BE_HP_FP_ADD_k2',
 '_Z12PowerKernal1PKdS0_Pdi': 'BE_DP_FP_ADD_k1',
 '_Z12PowerKernal1PKfS0_Pfi': 'BE_SP_FP_ADD_k1',
 '_Z12PowerKernal2PK6__halfS1_PS_i': 'BE_HP_FP_MUL_k2',
 '_Z12PowerKernal2PKdS0_Pdi': 'BE_DP_FP_MUL_k1',
 '_Z12PowerKernal2PKfS0_Pfi': 'BE_SP_FP_MUL_k1',
 '_Z12PowerKernal2PKjS0_Pji': 'BE_SP_INT_ADD_k1',
 '_Z17convertFp32ToFp16P6__halfPfi': 'BE_HP_FP_ADD_k1'}


In [8]:
os.chdir(results_dir)
df = pd.DataFrame.from_dict(power_dict, orient='index')
cwd = 'accelwattch_'+config+ '.csv'
df.to_csv(cwd)
os.chdir(rootdir)